In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

In [56]:
train = pd.read_csv('ins3_train.csv')
test = pd.read_csv('ins3_test.csv')

In [4]:
train.head()

,Customer Id,YearOfObservation,Insured_Period,Residential,Building_Painted,Building_Fenced,Garden,Settlement,Building Dimension,Building_Type,...,3,4,5,6,7,8,9,>=10,paint_fence,month
0,H14663,2013,1.0,0,1,0,1.0,1,290.0,1,...,0,0,0,0,0,0,0,0,1,12
1,H2037,2015,1.0,0,0,1,0.0,0,490.0,1,...,0,1,0,0,0,0,0,0,0,12
2,H3802,2014,1.0,0,1,0,1.0,1,595.0,1,...,0,0,0,0,0,0,0,0,1,12
3,H3834,2013,1.0,0,0,0,1.0,1,2840.0,1,...,0,0,0,0,0,0,0,0,0,12
4,H5053,2014,1.0,0,0,1,0.0,0,680.0,1,...,1,0,0,0,0,0,0,0,0,12


In [5]:
train.columns

Index(['Customer Id', 'YearOfObservation', 'Insured_Period', 'Residential',
       'Building_Painted', 'Building_Fenced', 'Garden', 'Settlement',
       'Building Dimension', 'Building_Type', 'Date_of_Occupancy', 'Geo_Code',
       'Claim', '   .', '1', '2', '3', '4', '5', '6', '7', '8', '9', '>=10',
       'paint_fence', 'month'],
      dtype='object')

In [57]:
test_id = test['Customer Id']

In [58]:
train = train.drop('Customer Id', axis=1)
test = test.drop('Customer Id', axis=1)

In [59]:
X = train.drop('Claim', axis=1)
y = train.Claim

In [60]:
# Split the data into 30% validation and 70% training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42) #0.15 bank40

In [61]:
from sklearn.metrics import auc

In [62]:
model=CatBoostClassifier(n_estimators=800,eval_metric='AUC',max_depth=5,learning_rate=0.1,od_wait=50, 
                              subsample=0.9,bootstrap_type='Bernoulli',metric_period=20,
                     #l2_leaf_reg=5,#bagging_temperature=0.85,random_strength=100,
                     use_best_model=True)

In [57]:
model.fit(X_train, y_train, eval_set=(X_test, y_test))

0:	test: 0.6844411	best: 0.6844411 (0)	total: 156ms	remaining: 2m 4s
20:	test: 0.7116009	best: 0.7116009 (20)	total: 2.96s	remaining: 1m 50s
40:	test: 0.7152016	best: 0.7153845 (38)	total: 4.65s	remaining: 1m 26s
60:	test: 0.7186523	best: 0.7188726 (58)	total: 6.08s	remaining: 1m 13s
80:	test: 0.7175294	best: 0.7194751 (66)	total: 7.43s	remaining: 1m 5s
100:	test: 0.7162682	best: 0.7194751 (66)	total: 8.84s	remaining: 1m 1s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.7194750883
bestIteration = 66

Shrink model to first 67 iterations.


In [58]:
pred = model.predict_proba(test)[:, 1]

In [63]:
# Split the data into 30% validation and 70% training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42) #0.15 bank40

In [83]:
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
errcb=[]
y_pred_totcb=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit, GroupKFold
fold=StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m=CatBoostClassifier(n_estimators=800,eval_metric='AUC',max_depth=4,learning_rate=0.1,#reg_lambda=5,#5
                              subsample=0.9,bootstrap_type='Bernoulli',#leaf_estimation_iterations=10,
                    #l2_leaf_reg=5,#bagging_temperature=0.85,random_strength=100,
                     use_best_model=True)
    #CatBoostClassifier(n_estimators=1000,eval_metric='AUC',max_depth=5,learning_rate=0.1,reg_lambda=5,#5
                              #subsample=0.9,bootstrap_type='Bernoulli',
                    #l2_leaf_reg=5,#bagging_temperature=0.85,random_strength=100,
                     #use_best_model=True)
    m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m.predict_proba(X_test)[:, 1]
    print("err: ",roc_auc_score(y_test,preds))
    errcb.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(test)[:, 1]
    y_pred_totcb.append(p)

0:	test: 0.6560506	test1: 0.6732939	best: 0.6732939 (0)	total: 127ms	remaining: 1m 41s
100:	test: 0.7495412	test1: 0.7380535	best: 0.7381511 (83)	total: 8.39s	remaining: 58.1s
200:	test: 0.7728954	test1: 0.7337153	best: 0.7383687 (109)	total: 15.7s	remaining: 46.6s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7383687456
bestIteration = 109

Shrink model to first 110 iterations.
err:  0.73836874558
0:	test: 0.6587901	test1: 0.6624167	best: 0.6624167 (0)	total: 113ms	remaining: 1m 30s
100:	test: 0.7592876	test1: 0.7165048	best: 0.7168539 (84)	total: 8.85s	remaining: 1m 1s
200:	test: 0.7791190	test1: 0.7117692	best: 0.7169023 (105)	total: 16.4s	remaining: 49s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7169022502
bestIteration = 105

Shrink model to first 106 iterations.
err:  0.716902250247
0:	test: 0.6724037	test1: 0.6643918	best: 0.6643918 (0)	total: 77ms	remaining: 1m 1s
100:	test: 0.7624397	test1: 0.7092100	best: 0.7098388 (42)	total: 

In [84]:
np.mean(errcb)

0.72170324868858182

In [252]:
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
errcb1=[]
y_pred_totcb1=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
fold=StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m1=CatBoostClassifier(n_estimators=1000,eval_metric='AUC',max_depth=4,learning_rate=0.1,od_wait=50, reg_lambda=3,
                              bootstrap_type='Bayesian',
                     use_best_model=True)
    m1.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m1.predict_proba(X_test)[:, 1]
    print("err: ",roc_auc_score(y_test,preds))
    errcb1.append(roc_auc_score(y_test,preds))
    p1 = m1.predict_proba(test)[:, 1]
    y_pred_totcb1.append(p1)

0:	test: 0.6747662	test1: 0.6829593	best: 0.6829593 (0)	total: 211ms	remaining: 3m 31s
100:	test: 0.7426594	test1: 0.7400637	best: 0.7401628 (91)	total: 9.69s	remaining: 1m 26s
200:	test: 0.7546493	test1: 0.7390750	best: 0.7411181 (127)	total: 16s	remaining: 1m 3s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7411180508
bestIteration = 127

Shrink model to first 128 iterations.
err:  0.741118050782
0:	test: 0.6750514	test1: 0.6740161	best: 0.6740161 (0)	total: 111ms	remaining: 1m 51s
100:	test: 0.7573646	test1: 0.7165387	best: 0.7166358 (60)	total: 8.34s	remaining: 1m 14s
200:	test: 0.7680030	test1: 0.7153891	best: 0.7168026 (155)	total: 16.1s	remaining: 1m 4s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7168026377
bestIteration = 155

Shrink model to first 156 iterations.
err:  0.716802637739
0:	test: 0.6791360	test1: 0.6578452	best: 0.6578452 (0)	total: 125ms	remaining: 2m 4s
100:	test: 0.7575013	test1: 0.7074546	best: 0.7091392 (22)	tot

In [253]:
np.mean(errcb1)

0.7223532904916542

In [87]:
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
fold=KFold(n_splits=5)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m2=XGBClassifier(max_depth=6, learning_rate=0.1, n_estimators=800, silent=True, metrics='auc',colsample_bylevel=0.8, reg_alpha=0.8)
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m2.predict_proba(X_test)[:, 1]
    print("err: ",roc_auc_score(y_test,preds))
    errcb2.append(roc_auc_score(y_test,preds))
    p2 = m2.predict_proba(test)[:, 1]
    y_pred_totcb2.append(p2)

[0]	validation_0-error:0.200943	validation_1-error:0.222765
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved in 100 rounds.
[100]	validation_0-error:0.165154	validation_1-error:0.208799
Stopping. Best iteration:
[7]	validation_0-error:0.192214	validation_1-error:0.206704

err:  0.718382339106
[0]	validation_0-error:0.198673	validation_1-error:0.215782
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved in 100 rounds.
[100]	validation_0-error:0.165154	validation_1-error:0.214385
Stopping. Best iteration:
[18]	validation_0-error:0.191515	validation_1-error:0.201117

err:  0.722738637471
[0]	validation_0-error:0.206878	validation_1-error:0.196927
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved in 100 r

In [88]:
np.mean(errcb2)

0.70628966172540253

In [254]:
d = {"Customer Id": test_id, 'Claim': np.mean(y_pred_totcb, 0)}
test_predictions = pd.DataFrame(data=d)
test_predictions = test_predictions[["Customer Id", 'Claim']]

In [255]:
#test_predictions.to_csv('in14.csv', index=False)
test_predictions.to_csv('in40.csv', index=False)

In [256]:
test_predictions.head()

,Customer Id,Claim
0,H11920,0.192289
1,H11921,0.183826
2,H9805,0.177754
3,H7493,0.242899
4,H7494,0.231618


In [93]:
d = {"Customer Id": test_id, 'Claim': np.mean(y_pred_totcb1, 0)}
test_predictioned = pd.DataFrame(data=d)
test_predictioned = test_predictioned[["Customer Id", 'Claim']]

In [94]:
test_predictioned.head()

,Customer Id,Claim
0,H11920,0.109179
1,H11921,0.101538
2,H9805,0.091374
3,H7493,0.161428
4,H7494,0.145549


In [95]:
d = {"Customer Id": test_id, 'Claim': np.mean(y_pred_totcb2, 0)}
test_prediction = pd.DataFrame(data=d)
test_prediction = test_prediction[["Customer Id", 'Claim']]

In [96]:
test_predictions.head()

,Customer Id,Claim
0,H11920,0.090933
1,H11921,0.084713
2,H9805,0.070089
3,H7493,0.148441
4,H7494,0.130436


In [257]:
a = pd.read_csv('in14.csv')
b = pd.read_csv('i1.csv')
c = pd.read_csv('in3.csv')
d = pd.read_csv('in40.csv')

In [258]:
d['Claim'].corr(a['Claim'])

0.97090875687873479

In [223]:
test_predictions['Claim'].corr(test_prediction['Claim'])

0.99901710195910698

In [271]:
test_predictions['Claim'] = (a['Claim'] * 0.69 + b['Claim'] * 0.31).round(2)

In [270]:
test_predictions.to_csv('ins_stack8.csv', index=False)